# GAMA-15 master catalogue
## Preparation of VIKING data

VISTA telescope/VIKING catalogue: the catalogue comes from `dmu0_VIKING`.

In the catalogue, we keep:

- The identifier (it's unique in the catalogue);
- The position;
- The stellarity;
- The magnitude for each band.
- The kron magnitude to be used as total magnitude (no “auto” magnitude is provided).

We don't know when the maps have been observed. We will use the year of the reference paper.

In [ ]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

In [ ]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

from collections import OrderedDict
import os

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np

from herschelhelp_internal.flagging import  gaia_flag_column
from herschelhelp_internal.masterlist import nb_astcor_diag_plot, remove_duplicates
from herschelhelp_internal.utils import astrometric_correction, mag_to_flux

In [ ]:
OUT_DIR =  os.environ.get('TMP_DIR', "./data_tmp")
try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

RA_COL = "viking_ra"
DEC_COL = "viking_dec"

## I - Column selection

In [ ]:
# Bands: Z,Y,J,H,K
imported_columns = OrderedDict({
        'SOURCEID': "viking_id",
        'ra': "viking_ra",
        'dec': "viking_dec",
        'PSTAR':  "viking_stellarity",
        'ZPETROMAG': "m_viking_z", 
        'ZPETROMAGERR': "merr_viking_z", 
        'ZAPERMAG3': "m_ap_viking_z", 
        'ZAPERMAG3ERR': "merr_ap_viking_z",
        'YPETROMAG': "m_viking_y", 
        'YPETROMAGERR': "merr_viking_y", 
        'YAPERMAG3': "m_ap_viking_y", 
        'YAPERMAG3ERR': "merr_ap_viking_y",
        'JPETROMAG': "m_viking_j", 
        'JPETROMAGERR': "merr_viking_j", 
        'JAPERMAG3': "m_ap_viking_j", 
        'JAPERMAG3ERR': "merr_ap_viking_j",        
        'HPETROMAG': "m_viking_h", 
        'HPETROMAGERR': "merr_viking_h", 
        'HAPERMAG3': "m_ap_viking_h", 
        'HAPERMAG3ERR': "merr_ap_viking_h",        
        'KSPETROMAG': "m_viking_k", 
        'KSPETROMAGERR': "merr_viking_k", 
        'KSAPERMAG3': "m_ap_viking_k", 
        'KSAPERMAG3ERR': "merr_ap_viking_k",
    })


catalogue = Table.read("../../dmu0/dmu0_VISTA-VIKING/data/VIKING_GAMA-15.fits")[list(imported_columns)]
for column in imported_columns:
    catalogue[column].name = imported_columns[column]

epoch = 2011

# Clean table metadata
catalogue.meta = None

In [ ]:
# Adding flux and band-flag columns
for col in catalogue.colnames:
    if col.startswith('m_'):
        
        errcol = "merr{}".format(col[1:])
        
        # Some object have a magnitude to 0, we suppose this means missing value
        catalogue[col][catalogue[col] <= 0] = np.nan
        catalogue[errcol][catalogue[errcol] <= 0] = np.nan  
        

        flux, error = mag_to_flux(np.array(catalogue[col]), np.array(catalogue[errcol]))
        
        # Fluxes are added in µJy
        catalogue.add_column(Column(flux * 1.e6, name="f{}".format(col[1:])))
        catalogue.add_column(Column(error * 1.e6, name="f{}".format(errcol[1:])))
        
        # Band-flag column
        if "ap" not in col:
            catalogue.add_column(Column(np.zeros(len(catalogue), dtype=bool), name="flag{}".format(col[1:])))
        
# TODO: Set to True the flag columns for fluxes that should not be used for SED fitting.

In [ ]:
catalogue[:10].show_in_notebook()

## II - Removal of duplicated sources

We remove duplicated objects from the input catalogues.

In [ ]:
SORT_COLS = ['merr_ap_viking_y', 'merr_ap_viking_h', 'merr_ap_viking_j', 'merr_ap_viking_k']
FLAG_NAME = 'viking_flag_cleaned'

nb_orig_sources = len(catalogue)

catalogue = remove_duplicates(catalogue, RA_COL, DEC_COL, sort_col=SORT_COLS,flag_name=FLAG_NAME)

nb_sources = len(catalogue)

print("The initial catalogue had {} sources.".format(nb_orig_sources))
print("The cleaned catalogue has {} sources ({} removed).".format(nb_sources, nb_orig_sources - nb_sources))
print("The cleaned catalogue has {} sources flagged as having been cleaned".format(np.sum(catalogue[FLAG_NAME])))

## III - Astrometry correction

We match the astrometry to the Gaia one. We limit the Gaia catalogue to sources with a g band flux between the 30th and the 70th percentile. Some quick tests show that this give the lower dispersion in the results.

In [ ]:
gaia = Table.read("../../dmu0/dmu0_GAIA/data/GAIA_GAMA-15.fits")
gaia_coords = SkyCoord(gaia['ra'], gaia['dec'])

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec)

In [ ]:
delta_ra, delta_dec =  astrometric_correction(
    SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]),
    gaia_coords
)

print("RA correction: {}".format(delta_ra))
print("Dec correction: {}".format(delta_dec))

In [ ]:
catalogue[RA_COL] +=  delta_ra.to(u.deg)
catalogue[DEC_COL] += delta_dec.to(u.deg)

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec)

## IV - Flagging Gaia objects

In [ ]:
catalogue.add_column(
    gaia_flag_column(SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]), epoch, gaia)
)

In [ ]:
GAIA_FLAG_NAME = "viking_flag_gaia"

catalogue['flag_gaia'].name = GAIA_FLAG_NAME
print("{} sources flagged.".format(np.sum(catalogue[GAIA_FLAG_NAME] > 0)))

## V - Flagging objects near bright stars

# VI - Saving to disk

In [ ]:
catalogue.write("{}/VISTA-VIKING.fits".format(OUT_DIR), overwrite=True)